In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import deque

In [2]:
from collections import defaultdict
np.random.seed(seed=111111)
np.set_printoptions(precision=3, suppress=True)

In [6]:
series3 = pd.read_csv("Series/mean-monthly-air-temperature-deg.csv")['Deg']

NameError: name 'pd' is not defined